In [1]:
import numpy as np
np.set_printoptions(precision=1)
import tensorflow as tf
import matplotlib.pylab as plt

from modules.utils import load_cifar10
from modules.cnn_with_spectral_parameterization import CNN_Spectral_Param
from modules.cnn_with_spectral_pooling import CNN_Spectral_Pool

% matplotlib inline
% load_ext autoreload
% autoreload 2

In [8]:
xtrain, ytrain, xtest, ytest = load_cifar10(5, channels_last=True)

file already downloaded..
getting batch 1
getting batch 2
getting batch 3
getting batch 4
getting batch 5


In [10]:
xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((50000, 32, 32, 3), (50000,), (10000, 32, 32, 3), (10000,))

In [11]:
# take a small batch for testing architecture
sampleind = np.random.choice(np.arange(xtrain.shape[0]),
                             size=64, replace=False)
xtrain_sample = xtrain[sampleind]
ytrain_sample = ytrain[sampleind]
assert len(np.unique(ytrain_sample)) == 10
print(np.bincount(ytrain_sample))

[ 8  8  4 11  3  6  7  7  3  7]


In [12]:
tf.reset_default_graph()
cnn = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=True)
cnn.train(xtrain_sample, ytrain_sample, None, None,
          batch_size=32, epochs=100)

number of batches for training: 2
epoch 1 
0.15625
7.1383
epoch 2 
0.125
6.57868
epoch 3 
0.25
5.6917
epoch 4 
0.09375
4.34716
epoch 5 
0.125
3.02162
epoch 6 
0.15625
2.92686
epoch 7 
0.375
2.63322
epoch 8 
0.46875
2.36253
epoch 9 
0.53125
2.4196
epoch 10 
0.65625
2.30791
epoch 11 
0.75
2.12652
epoch 12 
0.6875
1.92767
epoch 13 
0.78125
1.81962
epoch 14 
0.71875
1.76061
epoch 15 
0.84375
1.62036
epoch 16 
0.8125
1.56539
epoch 17 
0.8125
1.49091
epoch 18 
0.84375
1.45342
epoch 19 
1.0
1.37102
epoch 20 
1.0
1.32085
epoch 21 
0.9375
1.26383
epoch 22 
0.96875
1.21643
epoch 23 
1.0
1.16807
epoch 24 
1.0
1.13226
epoch 25 
1.0
1.10406
epoch 26 
1.0
1.06397
epoch 27 
1.0
1.03611
epoch 28 
1.0
1.01062
epoch 29 
1.0
0.987569
epoch 30 
1.0
0.959461
epoch 31 
1.0
0.934826
epoch 32 
1.0
0.916592
epoch 33 
1.0
0.89716
epoch 34 
1.0
0.881242
epoch 35 
1.0
0.866917
epoch 36 
1.0
0.852436
epoch 37 
1.0
0.836225
epoch 38 
1.0
0.824016
epoch 39 
1.0
0.813361
epoch 40 
1.0
0.801616
epoch 41 
1.0
0.790298


In [ ]:
tf.reset_default_graph()
cnn2 = CNN_Spectral_Param(num_output=10, kernel_size=5, architecture='generic', use_spectral_params=False)
cnn2.train(xtrain_sample, ytrain_sample, None, None,
          batch_size=32, epochs=100)

In [7]:
tf.reset_default_graph()
cnn3 = CNN_Spectral_Pool(M=9,
                        verbose=True,
                        learning_rate=1e-3,
                        alpha=0.25,
                        beta=1,
                        l2_norm=0.001,
                        lr_reduction_factor=0.5,
                        lr_reduction_epochs=[6,10,20,30,50,100],
                        gamma=0.85)

cnn3.train(xtrain_sample,
          ytrain_sample,
          xtrain_sample,
          ytrain_sample,
          batch_size=64,
          epochs=10,
          extra_conv_layer=True,
          use_global_averaging=True)

Building tf graph...
Adding conv layer 1 | Input size: 32 | Input channels: 3 | #filters: 128 | filter size: 3
Adding spectral pool layer 1 | Input size: 32 | filter size: (27,27) | Freq Dropout Bounds: (3.5,14.0)
Adding conv layer 2 | Input size: 27 | Input channels: 128 | #filters: 160 | filter size: 3
Adding spectral pool layer 2 | Input size: 27 | filter size: (21,21) | Freq Dropout Bounds: (2.75,11.0)
Adding conv layer 3 | Input size: 21 | Input channels: 160 | #filters: 192 | filter size: 3
Adding spectral pool layer 3 | Input size: 21 | filter size: (17,17) | Freq Dropout Bounds: (2.25,9.0)
Adding conv layer 4 | Input size: 17 | Input channels: 192 | #filters: 224 | filter size: 3
Adding spectral pool layer 4 | Input size: 17 | filter size: (13,13) | Freq Dropout Bounds: (1.75,7.0)
Adding conv layer 5 | Input size: 13 | Input channels: 224 | #filters: 256 | filter size: 3
Adding spectral pool layer 5 | Input size: 13 | filter size: (11,11) | Freq Dropout Bounds: (1.5,6.0)
Adding

InvalidArgumentError: Conv2DCustomBackpropFilterOp only supports NHWC.
	 [[Node: train_step/gradients/conv_layer_11/Conv2D_grad/Conv2DBackpropFilter = Conv2DBackpropFilter[T=DT_FLOAT, data_format="NCHW", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](conv_layer_10/Relu, train_step/gradients/conv_layer_11/Conv2D_grad/ShapeN:1, train_step/gradients/conv_layer_11/BiasAdd_grad/tuple/control_dependency)]]

Caused by op 'train_step/gradients/conv_layer_11/Conv2D_grad/Conv2DBackpropFilter', defined at:
  File "/Users/alexwainger/anaconda2/envs/dl/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/alexwainger/anaconda2/envs/dl/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/alexwainger/anaconda2/envs/dl/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/alexwainger/anacon